In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [22]:
data = tf.keras.datasets.fashion_mnist.load_data()

In [23]:
(x_train, y_train), (x_test, y_test) = data

In [24]:
x_train, x_test = tf.cast(x_train/255.0, tf.float32), tf.cast(x_test/255.0, tf.float32)

In [25]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [26]:
def preprocess_images(images):
    images = np.stack((images,) * 3, axis=-1)
    images = images.astype("float32") / 255.0
    return images
x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)
x_train_resized = tf.image.resize(x_train, [32, 32])
x_test_resized = tf.image.resize(x_test, [32, 32])

In [27]:
vgg = VGG16( weights="imagenet", include_top=False, input_tensor=layers.Input(shape=(32, 32, 3)))
for layer in vgg.layers:
    layer.trainable = True 
    
model_vgg = Sequential()
model_vgg.add(vgg)
model_vgg.add(Flatten())
model_vgg.add(Dense(256, activation="relu"))
model_vgg.add(Dropout(0.25))
model_vgg.add(Dense(10, activation="softmax"))
model_vgg.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 14848586 (56.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
model_vgg.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model_vgg.fit(x_train_resized, y_train, epochs=100, validation_data=(x_test_resized, y_test))

Epoch 1/10
  35/1875 [..............................] - ETA: 2:04:45 - loss: 2.3635 - accuracy: 0.1018

KeyboardInterrupt: 

In [ ]:
model_vgg.evaluate(x_test, y_test)